In [112]:
import pandas as pd

restaurants_data  = pd.read_csv('restaurants_clean.csv', encoding='utf8', index_col=0)

restaurants_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75334 entries, 0 to 1593
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            75334 non-null  object 
 1   city            75334 non-null  object 
 2   ranking         75334 non-null  float64
 3   rating          75334 non-null  float64
 4   price_range     75334 non-null  object 
 5   reviews_number  75334 non-null  int64  
 6   url_ta          75334 non-null  object 
 7   id_ta           75334 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 5.2+ MB


In [113]:
import sqlite3

conn = sqlite3.connect('restaurants_db.db')
cursor = conn.cursor()

cursor.execute('''
          DROP TABLE IF EXISTS RESTAURANT
          ''')

cursor.execute('''
          DROP TABLE IF EXISTS REVIEW
          ''')

cursor.execute('''
          DROP TABLE IF EXISTS CUISINE_STYLE
          ''')          

cursor.execute('''
          CREATE TABLE IF NOT EXISTS RESTAURANT
          ([id] INTEGER, [name] TEXT NOT NULL, [city] NOT NULL, [url_ta] TEXT, [ranking] INTEGER, [rating] numeric(8,2), [price_range] TEXT, [num_reviews] INTEGER, [id_ta] TEXT UNIQUE, PRIMARY KEY ([id], [id_ta]))
          ''')

cursor.execute('''
          CREATE TABLE IF NOT EXISTS REVIEW
          ([id] INTEGER NOT NULL PRIMARY KEY, [id_ta] TEXT NOT NULL, [comment] TEXT NOT NULL, [date] TEXT, FOREIGN KEY(id_ta) REFERENCES RESTAURANT(id_ta))
          ''')

cursor.execute('''
          CREATE TABLE IF NOT EXISTS CUISINE_STYLE
          ([id] INTEGER NOT NULL PRIMARY KEY, [id_ta] TEXT NOT NULL, [name] TEXT NOT NULL, FOREIGN KEY(id_ta) REFERENCES RESTAURANT(id_ta))
          ''')


conn.commit()

data=cursor.execute('''SELECT * FROM REVIEW''')
print(data.description)

#conn.close()

(('id', None, None, None, None, None, None), ('id_ta', None, None, None, None, None, None), ('comment', None, None, None, None, None, None), ('date', None, None, None, None, None, None))


In [114]:
#review_data  = pd.read_csv('reviews_table.csv', encoding='utf8', index_col=0)

#cuisine_data = pd.read_csv('cuisine_style_table.csv', encoding='utf8', index_col=0)

counter = 0
for row in restaurants_data.itertuples():
    cursor.execute('''
                INSERT OR IGNORE INTO RESTAURANT (id, name, city, url_ta, ranking, rating, price_range, num_reviews, id_ta)
                VALUES (?,?,?,?,?,?,?,?,?)
                ''',(
                counter, 
                row.name,
                row.city,
                row.url_ta,
                row.ranking,
                row.rating,
                row.price_range,
                row.reviews_number,
                row.id_ta)
                )
    counter+=1
conn.commit()

In [115]:
cursor.execute("select * from RESTAURANT")
t = cursor.fetchmany(5)
print(t)

[(0, "Martine of Martine's Table", 'Amsterdam', 'https://www.tripadvisor.com/Restaurant_Review-g188590-d11752080-Reviews-Martine_of_Martine_s_Table-Amsterdam_North_Holland_Province.html', 1, 5, '$$ - $$$', 136, 'd11752080'), (1, 'De Silveren Spiegel', 'Amsterdam', 'https://www.tripadvisor.com/Restaurant_Review-g188590-d693419-Reviews-De_Silveren_Spiegel-Amsterdam_North_Holland_Province.html', 2, 4.5, '$$$$', 812, 'd693419'), (2, 'La Rive', 'Amsterdam', 'https://www.tripadvisor.com/Restaurant_Review-g188590-d696959-Reviews-La_Rive-Amsterdam_North_Holland_Province.html', 3, 4.5, '$$$$', 567, 'd696959'), (3, 'Vinkeles', 'Amsterdam', 'https://www.tripadvisor.com/Restaurant_Review-g188590-d1239229-Reviews-Vinkeles-Amsterdam_North_Holland_Province.html', 4, 5, '$$$$', 564, 'd1239229'), (4, "Librije's Zusje Amsterdam", 'Amsterdam', 'https://www.tripadvisor.com/Restaurant_Review-g188590-d6864170-Reviews-Librije_s_Zusje_Amsterdam-Amsterdam_North_Holland_Province.html', 5, 4.5, '$$$$', 316, 'd68

In [116]:
import csv
   
with open('reviews_table.csv', encoding="utf8") as file_obj:
    counter = 0  
    reader_obj = csv.reader(file_obj)
      
    for row in reader_obj:
        if(len(row) == 3):
            cursor.execute('''
                    INSERT OR IGNORE INTO REVIEW (id, id_ta, comment, date)
                    VALUES (?,?,?,?)
                    ''',( 
                    counter,    
                    row[0],
                    row[1],
                    row[2])
                    )
            counter+=1
conn.commit()

In [117]:
cursor.execute("select * from REVIEW")
t = cursor.fetchmany(5)
print(t)

[(0, 'd4220190', 'Smile please!', '05/23/2017'), (1, 'd7618708', 'Slowest service in zurich', '02/07/2017'), (2, 'd7618708', 'Like Pret a Manager just not goof', '07/24/2017'), (3, 'd808215', 'Outside...not inside', '08/31/2017'), (4, 'd808215', 'Great Restaurant and Mexican food', '09/17/2017')]


In [118]:
import csv
   
with open('cuisine_style_table.csv', encoding="utf8") as file_obj:
    counter = 0  
    reader_obj = csv.reader(file_obj)
      
    for row in reader_obj:
        if(len(row) == 2):
            cursor.execute('''
                    INSERT OR IGNORE INTO CUISINE_STYLE (id, id_ta, name)
                    VALUES (?,?,?)
                    ''',( 
                    counter,    
                    row[0],
                    row[1])
                    )
            counter+=1
conn.commit()

In [119]:
cursor.execute("select * from CUISINE_STYLE")
t = cursor.fetchmany(5)
print(t)

[(0, 'd11752080', 'French'), (1, 'd11752080', 'Dutch'), (2, 'd11752080', 'European'), (3, 'd693419', 'Dutch'), (4, 'd693419', 'European')]


In [120]:
conn.close()